In [1]:
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import spacy
import textstat
from typing import List, Dict
import nest_asyncio

In [2]:
simple_replacements = {
    # Commonly used words
    "utilize": "use",
    "commence": "start",
    "terminate": "end",
    "ascertain": "find out",
    "endeavor": "try",
    "comprehend": "understand",
    "facilitate": "help",
    "subsequent": "next",
    "prior": "before",

    # Business and formal terms
    "accomplish": "do",
    "advantageous": "helpful",
    "adjacent": "next to",
    "allocate": "give",
    "ameliorate": "improve",
    "authorize": "allow",
    "collaborate": "work together",
    "commence": "begin",
    "constitutes": "is",
    "demonstrate": "show",
    "endeavor": "attempt",
    "expedite": "speed up",
    "implement": "carry out",
    "indicate": "show",
    "necessitate": "need",
    "procure": "get",
    "proficient": "skilled",
    "remuneration": "payment",
    "substantial": "large",
    "terminate": "stop",
    "validate": "confirm",

    # Academic terms
    "abbreviate": "shorten",
    "advocate": "support",
    "analyze": "study",
    "approximate": "almost",
    "articulate": "express",
    "categorize": "sort",
    "collate": "gather",
    "corroborate": "confirm",
    "elucidate": "explain",
    "enumerate": "list",
    "extrapolate": "guess",
    "facilitate": "help",
    "formulate": "create",
    "illustrate": "show",
    "implement": "do",
    "inquire": "ask",
    "interpret": "explain",
    "justify": "prove",
    "modify": "change",
    "subsequent": "later",
    "synthesize": "combine",

    # Technical and scientific terms
    "accelerate": "speed up",
    "accumulate": "gather",
    "adhere": "stick",
    "amplify": "increase",
    "compute": "calculate",
    "conceive": "imagine",
    "conduct": "carry out",
    "detect": "find",
    "disperse": "spread",
    "emit": "send out",
    "evaluate": "judge",
    "fluctuate": "change",
    "generate": "make",
    "illustrate": "show",
    "integrate": "combine",
    "isolate": "separate",
    "maximize": "increase",
    "minimize": "reduce",
    "optimize": "improve",
    "regulate": "control",
    "sustain": "maintain",
    "transmit": "send",

    # Everyday terms
    "commonly": "often",
    "endeavor": "try",
    "individual": "person",
    "necessitate": "require",
    "obtain": "get",
    "perceive": "see",
    "predominantly": "mostly",
    "substantial": "big",
    "sufficient": "enough",
    "transform": "change",
    "verify": "check",
}


In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
nest_asyncio.apply()

# Initialize the FastAPI app
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins for testing
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
class TextInput(BaseModel):
    text: str

# Function to suggest simpler words
def suggest_simpler_words(text: str) -> List[Dict[str, str]]:
    doc = nlp(text)
    suggestions = []
    for token in doc:
        if token.text.lower() in simple_replacements:
            suggestions.append({
                "original": token.text,
                "replacement": simple_replacements[token.text.lower()]
            })
    return suggestions

# Function to calculate readability
def calculate_readability(text: str):
    return textstat.flesch_reading_ease(text)

# Function to simplify text
def simplify_text(text: str):
    doc = nlp(text)
    simplified_tokens = []
    for token in doc:
        if token.text.lower() in simple_replacements:
            simplified_tokens.append(simple_replacements[token.text.lower()])
        else:
            simplified_tokens.append(token.text)
    return " ".join(simplified_tokens)

In [ ]:
# Define the POST endpoint
@app.post("/")
def analyze_text(input: TextInput):
    text = input.text
    doc = nlp(text)

    word_count = len([token for token in doc if token.is_alpha])
    char_count_without_spaces = len(text.replace(" ", ""))
    sentence_count = len(list(doc.sents))
    paragraphs = [p.strip() for p in text.split("\n") if p.strip()]
    paragraph_count = len(paragraphs)
    reading_time_minutes = word_count / 200
    readability_score = calculate_readability(text)
    suggestions = suggest_simpler_words(text)
    simplified_text = simplify_text(text)
    simplified_readability = calculate_readability(simplified_text)

    return {
        "word_count": word_count,
        "char_count_without_spaces": char_count_without_spaces,
        "sentence_count": sentence_count,
        "paragraph_count": paragraph_count,
        "reading_time_minutes": round(reading_time_minutes, 2),
        "readability_score": readability_score,
        "suggestions": suggestions,
        "simplified_text": simplified_text,
        "simplified_readability":simplified_readability
    }

# Define the GET endpoint
@app.get("/")
def read_root():
    return {"message": "Welcome to the Text Analysis API!"}

# Function to run the FastAPI server
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the FastAPI server
run_app()

INFO:     Started server process [1284]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     41.43.105.60:0 - "OPTIONS / HTTP/1.1" 200 OK
INFO:     41.43.105.60:0 - "POST / HTTP/1.1" 200 OK
INFO:     156.195.167.184:0 - "OPTIONS / HTTP/1.1" 200 OK
INFO:     156.195.167.184:0 - "POST / HTTP/1.1" 200 OK
